In [2]:
"""def padding(column: pd.Series) -> pd.Series:
    def pad_text(text, max_length=128): 
        return text[:max_length] if len(text) > max_length else text + text[:(max_length - len(text)) % len(text)]
    return column.apply(pad_text)

df["pad"] = padding(df["text"])
"""    


'def padding(column: pd.Series) -> pd.Series:\n    def pad_text(text, max_length=128): \n        return text[:max_length] if len(text) > max_length else text + text[:(max_length - len(text)) % len(text)]\n    return column.apply(pad_text)\n\ndf["pad"] = padding(df["text"])\n'

In [85]:
# maybe we can do better data augmentation with mixup -> we didnt use it so this cell part is just experimental
"""alpha = 0.3
l1 = temp3.iloc[0].values
l2 = temp3.iloc[3000].values
l = alpha*l1 + (1-alpha)*l2
print(l)






string2 = temp2["text"].iloc[1]
string3 = temp2["text"].iloc[-1]

mixed_string = string2[:int(alpha*len(string2))]+string3[:int((1-alpha)*len(string3))]
print(mixed_string)

temp = df["text"].sample(n=3, random_state=1)
df["text"]

print(temp2["lang"].head())"""

[0.3 0.  0.  0.7 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0. ]
61
90
طيب كم عدد المقاعد新片:[警員到犯罪現場強姦... -新片:[警員到犯罪現場強姦... -新片:[警員到犯罪現場強姦... -新片:[警員到犯
1066    ar
235     ar
1101    ar
909     ar
964     ar
Name: lang, dtype: object


# Extraction of our Dataset

In [191]:
# in this cell we tried to use deepl for translation since it was better but offered no free api use without the creation of an account with credit card
# we abandoned this approach and used google translate
"""
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait


options = webdriver.ChromeOptions()
options.add_argument("user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/89.0.4389.82 Safari/537.36")

browser = webdriver.Chrome(options=options)

#print(dir(browser))
browser.get("https://www.deepl.com/translator")

search = browser.find_element(By.CSS_SELECTOR,'.lmt__textarea_base_style')
search.clear()
search.send_keys("hola senorita muchas gracias")
WebDriverWait(browser, 2)
search2 = browser.find_element(By.XPATH,'//*[@id="panelTranslateText"]/div[1]/div[2]/section[1]/div[3]/div[2]/textarea')
#print(dir(search2))
print(browser.current_url)
print(search2.get_attribute("lang"))

find = browser.find_element(By.CSS_SELECTOR, ".lmt__inner_textarea_container , .lmt__textarea_base_style")
# //*[@id="panelTranslateText"]/div[1]/div[2]/section[2]/div[1]/div[1]/div[2]/button/span[2]/strong
f = browser.find_element(By.XPATH, '//*[@id="panelTranslateText"]/div[1]/div[2]/section[1]/div[1]/div[2]/button/span[2]/strong')
print(dir(f))
print(f.text)
driver.quit()"""

https://www.deepl.com/translator

['__abstractmethods__', '__class__', '__delattr__', '__dict__', '__dir__', '__doc__', '__eq__', '__format__', '__ge__', '__getattribute__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__le__', '__lt__', '__module__', '__ne__', '__new__', '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__sizeof__', '__str__', '__subclasshook__', '__weakref__', '_abc_impl', '_execute', '_id', '_parent', '_upload', 'accessible_name', 'aria_role', 'clear', 'click', 'find_element', 'find_elements', 'get_attribute', 'get_dom_attribute', 'get_property', 'id', 'is_displayed', 'is_enabled', 'is_selected', 'location', 'location_once_scrolled_into_view', 'parent', 'rect', 'screenshot', 'screenshot_as_base64', 'screenshot_as_png', 'send_keys', 'shadow_root', 'size', 'submit', 'tag_name', 'text', 'value_of_css_property']
Detect language


In [26]:
# sampling of tweets using the twitter api 
from inspect import getfullargspec
import tweepy
consumer_key = "xxx"
consumer_secret = "xxx"
access_token = "xxx"
access_token_secret = "xxx"
auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_token_secret)
api = tweepy.API(auth)
#print(inspect.signature(tweepy.Cursor))
cursor = tweepy.Cursor(api.home_timeline, tweet_mode="extended").items(1)
for i in cursor:
    print(i.full_text)
# Sample 100 random tweets
""""for tweet in tweepy.Cursor(api.search_tweets, q='*', lang='en').items(10):
    print(tweet.text)"""

Unexpected parameter: id


A single task that subsumes object detection and language understanding.
From #NYU. https://t.co/svCusDzJmM


'"for tweet in tweepy.Cursor(api.search_tweets, q=\'*\', lang=\'en\').items(10):\n    print(tweet.text)'

In [37]:
import pandas as pd 
import requests
import time
# we use google translate to improve the quality of our data 
def detect_language(text):
    url = "https://translate.googleapis.com/translate_a/single?client=gtx&sl=auto&tl=en&dt=t&q=" + text
    response = requests.get(url)
    json_response = response.json()
    return json_response[2]
counter = 0
"""for tweet in json.loads(response.text)["statuses"]:
    language = detect_language(tweet["text"])
    df["languages"].iloc[counter] = language
    counter = counter + 1
    time.sleep(0.1)
print(df["languages"].head(10))"""


lang = []
text = []
counter = 0
for tweet in tweepy.Cursor(api.search_tweets, q='*', lang='*').items(200):
    print(tweet.text)
    
    if tweet.lang == detect_language(tweet.text):
        counter = counter+1
        print(counter)
        lang.append(tweet.lang)
        text.append(tweet.text)


RT @yannosus: tEKA LANG NAMAN??!!!¿¿???@#£_

@SB19Official #SB19
#SB19_KEN https://t.co/Q14T48zVPi
RT @ricci_richy: This is the funniest Anne Curtis slip ever

*deadmahin nyo na lang yung asawa nung senador https://t.co/rNERKvUhaF
1
RT @nyangzun: #soobin was always someone who made his fans feel loved, so reading about how his perception of himsef as an artist changed t…
2
@3939hana 残念ながらはずれです…！
当たるまで毎日挑戦できます！

よろしければ、
メールでの事前登録もよろしくお願いします。
https://t.co/xrVFjZG0Bp

今後とも
『トランスフォーマー レイジバト… https://t.co/Xj134TSjR8
3
Tulog na nga lang kaagaw mo sa 'kin, nagagalit ka pa.
4
RT @TF_RAGE_BATTLE: ／
#TFレイバト #PV公開記念
フォロー＆RTキャンペーン開催
＼

抽選で10名様に1万円分の
#Amazonギフト券 当たる✨

 ■応募方法
①@TF_RAGE_BATTLE をフォロー
②このツイートをRT
③当選結果がその場…
RT @Shreverj: 'Total miscalculation': China goes into crisis management mode on balloon fallout #NewsBreak https://t.co/s1oEXHZo9V
5
RT @TIG_italia: Antonio Scarfò, Sales Manager Local Public Administration @Fortinet, è protagonista al Sardegna Digital Summit. Interviene…
6
Meu Círcu

In [28]:
zipped = list(zip(lang,text))
our_dataset = pd.DataFrame(zipped, columns = ["lang", "text"])
print(our_dataset.head(10))
final = our_dataset

  lang                                               text
0   en  RT @debwriting: I love that Xie Lian basically...
1   de  @heikki_seppala @derspiegel Der Leopard 1 ist ...
2   tl  sa mwcla pa lang na sinabi ni mari na hindi pi...
3   tl  de jk gumising lang talaga ko para sumagap ng ...
4   ja  @YoshihiroHonma3 残念ながらはずれです…！\n当たるまで毎日挑戦できます！\...
5   tl        madaling lang 💯 siraan 😮‍💨 mahirap sirain 💪
6   tl  Grabe now lang talaga nakumpleto for a dinner....
7   tl  tamang br ako sa mga tweet namin ni calvs 2020...
8   ko          미스딤플 화이트 란제리 토플리스 https://t.co/X7cg0q2rv9
9   ko  리샤 블랙허스트 누드 – Lishablackhurst 누드 사진 유출 https:/...


In [120]:
final.to_csv('C:/Users/n_bai/Documents/Praktikum/final.csv', index=False) 

In [29]:
print(our_dataset["lang"].value_counts())

tl    66
ja    39
ar    17
en    13
de    12
pt     7
ko     6
es     6
nl     2
ru     1
sl     1
da     1
Name: lang, dtype: int64
